# Use satellite magnetometer measurements to make a geomagnetic model


This is a notebook that produces a model of the Earth's magnetic field from magnetic field measurements taken by ESA's Swarm satellites. It is an attempt to do this in a way that is as simple as possible, but still gives good results. 

The model is based on a spherical harmonic representation of the magnetic field. That means that we must calculate the values of the spherical harmonic basis functions at the locations of the data, and at the locations where we want to evaluate the magnetic field. This is somewhat complicated (it includes for example, the calculation of Legendre functions), and the code for that is hidden in the sh module, which is imported below, along with other Python modules and scripts:

In [1]:
import numpy as np
from viresclient import SwarmRequest
import global_plots # tool for plotting data on the globe
import sh # tools for spherical harmonic analysis
import matplotlib.pyplot as plt
%matplotlib inline


We need to set some parameters for our model, most importantly the truncation level. That is where we stop the, in principle, infinite sum over spherical harmonics. Increasing the truncation levels (N and M) gives a higher resolution model, but it takes longer to fit, and may require more data. You can also adjust the number of data points that we use to estimate the model, but there is of course a limit for how much data we have. Experiment with changing these parameters. How little data can you use and still get good results?

The R parameters is just a scaling parameter, and it should not matter what value you choose as long as you use the same value when you estimte the model parameters and when you calculate model predictions. In geomagnetic modeling, it is traditionally set to 6371.2 km, which is roughly the average Earth radius. 

In [2]:
N_data = 500 # number of data points that I will extract
N, M = 12, 12 # spherical harmonic degree and order
R    = 6371.2e3 # the Earth radius in m

Below we download Swarm data. We use the viresclient python module to do that. Try experimenting with different time periods (Swarm data is available from 2014 and onward), and see if you can detect changes in the magnetic field. You can also experiment with filtering the data by other parameters. For example, it is common to use only data from geomagnetically quiet times to reduce the effect of solar wind-induced disturbances in the magnetic field. It is also common to only use data taken at the Earth's nightside to avoid magnetic disturbances associated with sunlight-induced currents in the ionosphere. 

I grab data only from Swarm A and B, even though there is a third satellite, Swarm C. This is because Swarm A and Swarm C fly side by side, and measure basically the same field (for our purposes). So including C doesn't give us more information. I also grab data only every 30 seconds, even though measurements are available at 1 s resolution. The reason for this is that two consecutive measurements are taken so close together (about 7.5 km, since the satellites move at 7.5 km/s) that they do essentially not contribute new information about the structure of the magnetic field at the resolutions we are interested in. 

In [ ]:
request = SwarmRequest()

# choose data from swarm A and B - we skip C since it is flying right next to A
request.set_collection('SW_OPER_MAGA_LR_1B', 'SW_OPER_MAGB_LR_1B')

# choose vector magnetic measurements every 30 seconds + some auxiliary measurement that we will use for filtering
# B_NEC means B (magnetic field) in the North East and Centre (radially inward) directions. 
request.set_products(measurements=['B_NEC'], auxiliaries=['Kp'], sampling_step='PT30S')

# set filter
pass

# get data from January 2020:
data = request.get_between('2020-01-01T00:00:00', '2020-02-01T00:00:00')

# save the data as a pandas dataframe 
df = data.as_dataframe()

print('We have {} data points'.format(len(df)))

[1/1] Processing:  100%|███████████████████████████████████████████████████████████████████████████████████████|  [ Elapsed: 00:03, Remaining: 00:00 ]
      Downloading:  30%|██████████████████████▌                                                     |  [ Elapsed: 00:07, Remaining: 00:18 ] (11.469MB)

Now we have a lot of data, but I'm not going to use everything. One reasons for this is that the data points are not uniformly distributed on the sphere. There is much more data per square km near the poles than near equator. If we don't do anything with this, our model will be more strongly determined by the measurements at the poles, and may not fit the field near the equator so well. We want a *global* model, so we have to fix it somehow. 

The reason that we have more data near the pole is that the satellites are in a polar orbit, and visit both poles every orbit (one orbit takes about 1.5 hour). On the other hand, they visit a specific place near the equator only once per day, since the orbital planes are roughly fixed with respect to the Sun, while the Earth rotates. To get an approximately uniform distribution of data points I choose a random subset using a specific weight function that depends on latitude. I plot two examples of subsets, with and without the weighting. 

Normally, we would not do it in exactly this way, but instead use a weighting function in the inversion step (see further down). But I think this method is more illustrative, and I want to keep the inversion as simple as possible.  


In [ ]:
weight = np.sin( (90 - df['Latitude']) * np.pi / 180)
p = weight / np.sum(weight)

indices = np.random.choice(np.arange(len(df)), N_data, replace = True, p = p) 
subset = df.iloc[indices] 

# plot the result
global_plots.global_scatterplot(subset.Latitude, subset.Longitude, np.ones(len(subset)), vmax=1, vmin=0.2, title = 'Uniform distribution of data points')

# just to illustrate, see what happens if we don't use the weighting:
indices = np.random.choice(np.arange(len(df)), N_data, replace = True) 
_ = df.iloc[indices] 
global_plots.global_scatterplot(_.Latitude, _.Longitude, np.ones(len(subset)), vmax=1, vmin=0.2, title = 'Unweighted')
print('done')

Now we have our dataset, and can set up the system of equations that relate Gauss coefficients (scaling factors in the spherical harmonic expansion) to the data. In the three lines below, we calculate $G$, $d$ and $m$ in the matrix equation 

$Gm = d$

Here, $m$ is a vector that contains the (K) Gauss coefficients (the scaling factors in the spherical harmonic representation of the magnetic field). $d$ is a vector that contains all the measured magnetic field components. It is 3 times N_data long, and composed of first the northward magnetic field components, then the eastward components, and then the components of the field that point radially inward. The $G$ matrix is a 3N_data $\times$ K matrix that relates the data points to the Gauss coefficients using the equations for spherical harmonics.

The $m$ vector, the solution vector, is found my minimizing the squared error. That is what the np.linalg.lstsq function does. So we use a least squares method, which should be familiar. There are of course more advanced ways to do this, but it works. 

Once we have the solution vector, I extract the coefficients that correspond to cos terms (I call those coefficients g) and sin terms (called h). I also extract the three coefficients that corresponds to the dipole part of the field. 


In [ ]:
# now we are ready to set up the system of equations (this gets complicated):
G, cos_keys, sin_keys = sh.get_G(subset['Radius'].values, subset['Latitude'].values, subset['Longitude'].values, N, M)

d = np.hstack(np.vstack(subset['B_NEC'].values).T)

# now we fit the cofficents
m = np.linalg.lstsq(G, d, rcond = 0)[0]

# the G matrix is organized with the cos coefficients (g) first and then the sin coefficients (h):
g = m[: len(cos_keys)]
h = m[len(cos_keys) :]

# get the dipole coefficients;
g10, g11 = g[:2]
h11 = h[0]



Now we plot the misfit (difference between data and our model):

In [ ]:

# plot the misfit as function of latitude:
dm = G.dot(m) # model predictions
error = dm - d
err_n, err_e, err_c = np.split(error, 3)
fig, ax = plt.subplots(figsize = (15, 8))
ax.scatter(subset['Latitude'].values, err_n, label = 'North')
ax.scatter(subset['Latitude'].values, err_e, label = 'East')
ax.scatter(subset['Latitude'].values, err_c, label = 'Center')
ax.legend(frameon = False)
ax.set_xlabel('Latitude')
ax.set_ylabel('Misfit [nT]')



And then a plot of the model predictions of the radial magnetic field component, evaluated on a global grid. Compare this to a scatter plot of the data points that we used to make the model

In [ ]:
lat, lon = np.meshgrid(np.linspace(-89.9, 89.9, 90), np.linspace(0, 360, 190)) # grid for plotting
G_grid, _, __ = sh.get_G(np.full(lat.size, R), lat.flatten(), lon.flatten(), N, M)
BC_model = np.split(G_grid.dot(m), 3)[2]

global_plots.global_contour(lat.flatten(), lon.flatten(), BC_model, vmin = -60000, vmax = 60000)
print('')

Compare to data values:

In [ ]:
global_plots.global_scatterplot(subset.Latitude, subset.Longitude, np.split(d, 3)[2], vmax=60000, vmin=-60000) # Br
print('')

Now we print the dipole pole locations, and dipole coefficients. We can compare our dipole model to the International Geomagnetic Reference Field (IGRF) model, which is a much used model of the Earth's magnetic field:

In [ ]:
B0 = np.sqrt(g10**2 + g11**2 + h11**2)
pole_lat, pole_lon = 90 - np.arccos(-g10/B0) / np.pi * 180, np.arctan2(-h11, -g11) / np.pi * 180
print('The location of my dipole pole is {:.2f} degrees north, {:.2f} degrees east'.format(pole_lat, pole_lon))
print('g10 = {:.1f}, g11  = {:.1f}, h11 = {:.1f}'.format(g10, g11, h11))
print('The corresponding IGRF 2020 coefficients are -29404.8, -1450.9, and 4652.5')

Calculate and print the magnetic declination at various points 

Note: Here we treat the Earth as spherical. Usually when you see coordinates of a place on Earth, it refers to a *geodetic* system, which means that the ellipsoidal shape of the planet is taken into account. So If you insert those coordinates in the code below, you will be slightly off where you want to be. 

In [ ]:
lats = np.array([55.5, 73])
lons = np.array([20.0, 30])

G_points, _, __ = sh.get_G(np.full(lats.size, R), lats, lons, N, M)
BN, BE, BC = np.split(G_points.dot(m), 3)

for i in range(len(lats)):
    declination = np.arctan(BE[i]/BN[i]) * 180 / np.pi
    print('The magnetic declination at ({:.1f} N, {:.1f} E) is {:.2f} degrees'.format(lats[i], lons[i], declination))
    